In [1]:
from pyplasm import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.005185 seconds


# Homework 02 - Alessandro Oddi

## Immagine di Riferimento

![imagine](http://systemgvp.altervista.org/images/immagini/lavori/struttura_3.jpg)

## Codice per creare il singolo reticolo da File csv

In [2]:
def createBeamsAndPillars(beamSec,pillarSec,distanceBtwPill,eightsBeam):
    maxHeight= SUM(eightsBeam)+beamSec[1]
    pillarLong=CUBOID([pillarSec[0],pillarSec[1],maxHeight]) 
    pillars=[]
    floor=[]
   
    for d in range(len(distanceBtwPill)):
        pillars.append(pillarLong)
        floor.append(T(2)(pillarSec[1]))
        
        for h in range(0,len(eightsBeam)):
            beam=CUBOID([beamSec[0],distanceBtwPill[d],beamSec[1]])
            floor.append(T(3)(eightsBeam[h]))
            floor.append(beam)
       
    
        floor.append(T(2)(pillarSec[1]))  
        img=STRUCT(floor) 
        floor=[]
        pillars.append(img)
        img=[]
        pillars.append(T(2)(distanceBtwPill[d]+pillarSec[1]))
        
    
    pillars.append(pillarLong)
    final=STRUCT(pillars)
    return final

## Metodo ausiliario richiesto nell'HW 01

In [3]:
def ggpl_bone_structure(file_name):
    
    beamAndPillar=[]
    translation=[]
    odd=True
    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            if(odd):
                x=float(row[0])
                y=float(row[1])
                z=float(row[2])
                tripla=(x,y,z)
                translation.append(tripla)
                odd=False
                
            else:
                element1= (float(row[0]),float(row[1]))
                element2= (float(row[2]),float(row[3]))
                element3=[]
                element4=[]
                cont1=int(row[4])
                for i in range(0,cont1):
                    element3.append(float(row[5+i]))
                cont2=int(row[5+cont1])
                for i in range(0,cont2):
                    element4.append(float(row[6+cont1+i]))
                
                obj=[]
                obj.append(element1)
                obj.append(element2)
                obj.append(element3)
                obj.append(element4)
                beamAndPillar.append(obj)
                
                odd=True
                      
    return combineStructAndTranslation(beamAndPillar,translation)


## Metodo che combina traslazioni e Reticoli
Raccolte le informazioni dal file csv si procede trasnlando i reticoli e concatenandoli grazie al metodo che segue

In [4]:
def combineStructAndTranslation(beamAndPillar,translation):
    building=[]
    for i in range(0,len(translation)):
        vx=translation[i][0]
        vy=translation[i][1]
        vz=translation[i][2]
    
        building.append(T(1)(vx))
        building.append(T(2)(vy))
        building.append(T(3)(vz))
        frame=beamAndPillar[i]

        building.append(createBeamsAndPillars(frame[0],frame[1],frame[2],frame[3]))

    img=STRUCT(building)
    
    
    return connectFrames(img,beamAndPillar,translation)
    

## Gestione delle Travi che uniscono i Reticoli
Si entra in un loop sulle traslazioni: finche c'è una traslazione vengono generate travi.
Le travi sono generate gravie ai vettori x,y,z, cosi formati:

* x = (larghezza su x del pilastro, distanza traslazione sul piano x)
* y = (0, larghezza su y del pilastro)
* z = (altezza del pilastro corrente, dimenzione su z del pilastro)

Vengono gestite solamente le strutture con connessioni ortogonali. In questo codice non viene utilizzata la funzione QUOTE per gestire la "parametrizzazione" delle travi rispetto alla lunghezza delle liste di distanze su y e z.

In [ ]:
def connectFrames(img,beamAndPillar,translation):
    
    beamConnection=[]
    translateSum=0
    vx=translation[0][0]
    vy=translation[0][1]
    vz=translation[0][2]
    beamConnection.append(T(1)(vx))
    beamConnection.append(T(2)(vy))
    beamConnection.append(T(3)(vz))
    
    for t in range(1,len(translation)):
        
        floorSum=0
        frame=beamAndPillar[t]
        for floor in frame[3]:
            x=(-frame[1][0],translation[t][0]-frame[1][0])
            y=(0,frame[1][1])
            floorSum=floorSum+floor
            z=(-floorSum,frame[0][1])
            a=QUOTE(x*1)
            b=QUOTE(y*1)
            c=QUOTE(z*1)
            aa=PROD([a,b])
            aaa=PROD([aa,c])
            for distance in frame[2]:
                beamConnection.append(aaa)
                beamConnection.append(T(2)(distance+frame[1][1]))
                allDistance=distance+frame[1][1] 
            beamConnection.append(aaa)
            allDistance=allDistance+distance
            if(vx!=0 and vy!=0):
                beamConnection.append(T(2)(-allDistance+vx))
            if(vx!=0 and vy==0):
                beamConnection.append(T(2)(-(allDistance/2)-(frame[1][1]/2)))
            if(vx==0 and vy==0):
                beamConnection.append(T(2)(-allDistance-frame[1][1])) 
                
        beamConnection.append(T(1)(translation[t][0]))
                               
    beamConnection=STRUCT(beamConnection)  
    img2=STRUCT([img,beamConnection])  
    
    return img2

## Creazione della Struttura
Al fine di realizare una struttura simile a quella di riferimento ho scritto tre diversi file csv in cui venivano descritti le tre componenti principali della mia struttura: a,b,c.
La scelta di suddividere i file csv è dovuta al fatto che la mia struttura di riferimento presenta una sezione concava; quindi, per non rendere ancora più complicato un codice già complicato di suo, mi è risultato più semplice suddividere le fondamenta in tre componenti, e poi ri-accorparle insieme una volta create.

In [ ]:
a=ggpl_bone_structure("framedata_461980.csv")
b=ggpl_bone_structure("framedata_461980pt1.csv")
c=ggpl_bone_structure("framedata_461980pt2.csv")
abc=[]
abc.append(a)
abc.append(b)
abc.append(c)
abc=STRUCT(abc)
VIEW(abc)
return abc

## Risultati e Conclusioni
Seguono gli screen del risultato ottenuto che, eccezione fatta per il numero di piani volutamente aumentato, rispetta appienno il modello e e le proporzioni della struttura in cemento armato scelta come riferimento (http://systemgvp.altervista.org/images/immagini/lavori/struttura_3.jpg)

### Conclusioni
Per quanto riguarda il codice in sé per sé, sono pienamente convinto si potesse fare di meglio, sia dal punto di vista della complessita, sia per liberarsi dei cicli e per utilizzare unicamente le funzioni QUOTE e PRODU nella generazione delle travi. 
Spero nelle prossime settimane di riuscire a implementare la funzione ***connectFramesV2(img,beamAndPillar,translation)*** che produca gli stessi risultati senza l'utilizzo di molti cicli
  
### Screen del risultato
![risultato 1](https://s21.postimg.org/oky0ssqh3/Schermata_2016_10_22_alle_17_10_02.png)
![risultato 2](https://s21.postimg.org/lfdf2l7uv/Schermata_2016_10_22_alle_17_09_45.png)
![risultato 3](https://s21.postimg.org/hka100op3/Schermata_2016_10_22_alle_17_06_55.png)

In [ ]:
def connectFramesV2(img,beamAndPillar,translation):
    #da implementare per migliorare l'efficenza
    return 0